**Read Data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('new.csv', names=["Timestamp", "Duration", "Type", "Level",
                                      "Client", "Client ID", "Query ID",
                                      "Query Name", "View", "Recursion",
                                      "Query Type", "Query", "Class",
                                      "Record Type", "Flags", "IP Address"],low_memory=False)
df.head()

**Check dataset property**

In [ ]:
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

null_values = df.isnull().sum()
print("Null values :\n",null_values)

In [ ]:
# Displaying rows with any null values
rows_with_nulls = df[df.isnull().any(axis=1)]
print(rows_with_nulls.head())

**Data Cleaning**

In [ ]:
# Dropping null values
df.dropna(inplace=True)

# Verify that the rows with null values have been dropped
print(df.isnull().sum())
print(df.shape)

**Feature selection**

In [ ]:
# List of columns to select
columns_to_select = ["Timestamp", "Duration", "Client", "Client ID", "Query ID", "Query", "Class", "Record Type", "Flags", "IP Address"]

# Select the specified columns
selected_df = df[columns_to_select]

# Display the first few rows of the selected DataFrame
print(selected_df.head())

**Transforming**

In [ ]:
# Convert the "Timestamp" column to datetime with error handling
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Verify the conversion
print(df['Timestamp'].head())
print(df['Timestamp'].dtype)

In [ ]:
df.info()

**Date time stamp**

In [ ]:
import pandas as pd

# Define the function to format time
def format_time(time_str):
  try:
      # Split the time string into minutes, seconds, and tenths of a second
      minutes, rest = time_str.split(':')
      seconds, tenths = rest.split('.')

      # Convert to integers
      minutes = int(minutes)
      seconds = int(seconds)
      milliseconds = int(tenths) * 100  # Convert tenths of a second to milliseconds

      # Calculate total milliseconds
      total_milliseconds = (minutes * 60 * 1000) + (seconds * 1000) + milliseconds

      # Convert to timedelta
      time_delta = pd.to_timedelta(total_milliseconds, unit='ms')

      # Format the time
      formatted_time = f"{time_delta.components.hours:02}:{time_delta.components.minutes:02}:{time_delta.components.seconds:02}.{time_delta.components.milliseconds:03}"
      return formatted_time
  except Exception as e:
      print(f"Error formatting time: {time_str} - {e}")
      return None  # Return None for invalid entries

# Create a copy of the DataFrame
selected_columns = selected_df.copy()

# Apply the function to each row
selected_columns['time'] = selected_columns['Duration'].apply(format_time)

# Convert the 'time' column to timedelta type
selected_columns['time'] = pd.to_timedelta(selected_columns['time'])

# Format the 'time' column to remove "0 days"
selected_columns['time'] = selected_columns['time'].apply(lambda x: f"{x.components.hours:02}:{x.components.minutes:02}:{x.components.seconds:02}.{x.components.milliseconds:03}")

# Print the updated DataFrame
print(selected_columns.head())

**Selecting and dropping feature column**

In [ ]:
selected_columns = selected_columns.drop('Duration', axis=1)
print(selected_columns.head())

In [ ]:
# Combine 'Timestamp' and 'time' columns into a single datetime column
selected_columns['Timestamp'] = pd.to_datetime(selected_columns['Timestamp']) + pd.to_timedelta(selected_columns['time'])

# Drop the 'time' column as it's now incorporated into 'Timestamp'
selected_columns = selected_columns.drop('time', axis=1)

# Print the updated DataFrame
print(selected_columns.head())

In [ ]:
selected_columns.info()

**Regex and query patterns**

In [ ]:
import re
import pandas as pd

# Define the regular expressions
regex_patterns = {
  "service_specific_dns": r"^_ldap\._tcp\.dc\._msdcs\.[A-Z]+\.[A-Z]+$",
  "uuid_based_service_specific_dns": r"^_ldap\._tcp\.[0-9a-fA-F-]{36}\.domains\._msdcs\.[A-Z]+\.[A-Z]+$",
  "standard_hostname_domain": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",
  "complex_subdomain_structure": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",
  "standard_hostname_subdomain": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",
  "simple_domain": r"^[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Added to match simple domains like "amazon.com"
  "single_label_domain": r"^[a-zA-Z0-9-]+$",  # Added to match single label domains like "."
  "extended_subdomain_structure": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Extended subdomain structure
  "another_extended_subdomain_structure": r"^[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Another extended subdomain structure
  "ip_based_subdomain": r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # IP-based subdomain
  "complex_ip_based_subdomain": r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Complex IP-based subdomain
  "subdomain_with_underscore": r"^[a-zA-Z0-9-_]+\.[a-zA-Z0-9-]+\.[a-zA-Z0-9-]+\.[a-zA-Z]{2,}$",  # Subdomain with underscore
  "complex_subdomain_with_underscore": r"^[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+\.[a-zA-Z]{2,}$"  # Complex subdomain with underscore
}


# Extract query names
query_names = selected_columns['Query'].tolist()

# Function to match query names to regex patterns
def match_query_names(query_names, regex_patterns):
  matches = {}
  for name in query_names:
      for pattern_name, pattern in regex_patterns.items():
          if re.match(pattern, name):
              matches[name] = pattern_name
              break
  return matches

# Match the query names
matched_queries = match_query_names(query_names, regex_patterns)

# Print the results
for query, pattern_name in matched_queries.items():
  print(f"Query: {query} matches pattern: {pattern_name}")



**IP Address and Time stamp duration**

In [ ]:
"""import pandas as pd

# Clean up the 'IP Address' column by removing parentheses
# str.replace(r'[()]', '', regex=True) removes both opening and closing parentheses in one go
selected_columns['IP Address'] = selected_columns['IP Address'].str.replace(r'[()]', '', regex=True)

# Set the 'Timestamp' column as the index and ensure it's in datetime format
selected_columns['Timestamp'] = pd.to_datetime(selected_columns['Timestamp'])
#selected_columns.set_index('Timestamp', inplace=True)

# Get unique IP addresses in the DataFrame
unique_ip_addresses = selected_columns['IP Address'].unique()

# Iterate through each unique IP address
for ip_address in unique_ip_addresses:
  # Filter the DataFrame by the current IP address
  filtered_df = selected_columns[selected_columns['IP Address'] == ip_address]

  # Ensure the DataFrame is sorted by index (timestamp) before resampling
  filtered_df = filtered_df.sort_index()

  # Resample the data to the desired time intervals and count the number of queries
  resampled_30min = filtered_df.resample('30T').size()
  resampled_1hour = filtered_df.resample('1H').size()
  resampled_1day = filtered_df.resample('1D').size()
  resampled_3days = filtered_df.resample('3D').size()

  # Combine the results into a DataFrame
  results = pd.DataFrame({
      '30min': resampled_30min,
      '1hour': resampled_1hour,
      '1day': resampled_1day,
      '3days': resampled_3days
  }).fillna(0).astype(int)

  # Print the results for the current IP address
  print(f"\nResampled results for IP address {ip_address}:")
  print(results)"""

**Using regex extract domain name**

In [ ]:
# Define a regex pattern to extract the domain name
# This pattern captures the domain name and its extension
domain_pattern = r'^(?:https?://)?(?:www\.)?([^/]+)'

# Extract the domain name from the 'Query' column
selected_columns['domain'] = selected_columns['Query'].str.extract(domain_pattern)

# Print the DataFrame to see the new 'domain' column
selected_columns

**Read Domain_label.txt**

In [ ]:
domain_data=pd.read_csv("domain_labels.txt",sep="\t")
domain_data.head()

**Read data from each column**

In [ ]:
# Read data from the text file
with open('domain_labels.txt', 'r') as file:
    lines = file.readlines()
data = []
for line in lines:
    line = line.strip()
    parts = line.split(',') + [""] * (4 - len(line.split(',')))
    data.append(parts)
column1, column2, column3, column4 = zip(*data)
print("Column 1:", list(column1))
print("Column 2:", list(column2))
print("Column 3:", list(column3))
print("Column 4:", list(column4))

**Display text file data as dataFrame**

In [ ]:
data_1=pd.DataFrame({"domain_name":list(column1),"categorized":list(column2),"category":list(column3),"risk":list(column4)})#.map({"False":0,"True":1,"Benign":0,"Low":1})
data_1

**Map and replace values**

In [ ]:
print(data_1["categorized"].unique())
data_1["categorized"]=data_1["categorized"].replace({"False":0,"True":1})

print(data_1["category"].unique())
data_1["category"]=data_1["category"].replace({"Benign":1})

print(data_1["risk"].unique())
data_1["risk"]=data_1["risk"].replace({"Low":0})

data_1.head()

**Final Dataset**

In [ ]:
selected_columns.join(data_1)

In [ ]:
import pandas as pd

# Clean up the 'IP Address' column by removing parentheses
# str.replace(r'[()]', '', regex=True) removes both opening and closing parentheses in one go
selected_columns['IP Address'] = selected_columns['IP Address'].str.replace(r'[()]', '', regex=True)

# Set the 'Timestamp' column as the index and ensure it's in datetime format
selected_columns['Timestamp'] = pd.to_datetime(selected_columns['Timestamp'])
selected_columns.set_index('Timestamp', inplace=True)

# Get unique IP addresses in the DataFrame
unique_ip_addresses = selected_columns['IP Address'].unique()

# Iterate through each unique IP address
for ip_address in unique_ip_addresses:
  # Filter the DataFrame by the current IP address
  filtered_df = selected_columns[selected_columns['IP Address'] == ip_address]

  # Ensure the DataFrame is sorted by index (timestamp) before resampling
  filtered_df = filtered_df.sort_index()

  # Resample the data to the desired time intervals and count the number of queries
  resampled_30min = filtered_df.resample('30T').size()
  resampled_1hour = filtered_df.resample('1H').size()
  resampled_1day = filtered_df.resample('1D').size()
  resampled_3days = filtered_df.resample('3D').size()

  # Combine the results into a DataFrame
  results = pd.DataFrame({
      '30min': resampled_30min,
      '1hour': resampled_1hour,
      '1day': resampled_1day,
      '3days': resampled_3days
  }).fillna(0).astype(int)

  # Print the results for the current IP address
  print(f"\nResampled results for IP address {ip_address}:")
  print(results)

In [ ]:
selected_columns.columns

**LSTM model**

**1. Importing libraries**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt


**2. Load data feature column**

In [ ]:
# Load the DataFrame (assuming you have the data in a CSV file)


# Select the relevant columns
selected_columns = selected_columns[['Client', 'Client ID', 'Query ID', 'Query', 'Class', 'Record Type', 'Flags', 'IP Address', 'domain']]

# For this example, we'll use 'Query ID' as the target to predict
# You can choose any other numerical column if needed


**3. Data Preprocessing**

In [ ]:
# Check and remove non-numeric values from 'Query ID'
selected_columns['Query ID'] = pd.to_numeric(selected_columns['Query ID'], errors='coerce')
selected_columns = selected_columns.dropna(subset=['Query ID'])

# Scale the 'Query ID' column (or any other numerical column you choose to predict)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(selected_columns['Query ID'].values.reshape(-1, 1))

# Convert the DataFrame to a format suitable for LSTM
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Define the number of previous steps to consider for prediction (e.g., 60)
time_step = 60

# Create the dataset for training
X, y = create_dataset(scaled_data, time_step)

# Reshape the input to be [samples, time steps, features] as required for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


**4. Build the LSTM Model**

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


**5. Train the Model**

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10)


**6. Make Predictions and Visualize the Results**

In [ ]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform to get actual values
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot the results
plt.figure(figsize=(14,7))
plt.plot(range(len(y_train_actual)), y_train_actual, color='blue', label='Training data')
plt.plot(range(len(y_train_actual), len(y_train_actual) + len(y_test_actual)), y_test_actual, color='green', label='Testing data')
plt.plot(range(len(y_train_actual)), train_predict, color='red', label='Training prediction')
plt.plot(range(len(y_train_actual), len(y_train_actual) + len(y_test_actual)), test_predict, color='orange', label='Testing prediction')
plt.title('Query ID Prediction')
plt.xlabel('Index')
plt.ylabel('Query ID')
plt.legend()
plt.show()


**7. Evaluation Metrics**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate metrics
mse_train = mean_squared_error(y_train_actual, train_predict)
mse_test = mean_squared_error(y_test_actual, test_predict)

mae_train = mean_absolute_error(y_train_actual, train_predict)
mae_test = mean_absolute_error(y_test_actual, test_predict)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

r2_train = r2_score(y_train_actual, train_predict)
r2_test = r2_score(y_test_actual, test_predict)

# Print metrics
print("Training : ")
print(f"Train MSE: {mse_train}")
print(f"Train MAE: {mae_train}")
print(f"Train RMSE: {rmse_train}")
print(f"Train R^2: {r2_train}")
print("\nTesting : ")
print(f"Test MSE: {mse_test}")
print(f"Test MAE: {mae_test}")
print(f"Test RMSE: {rmse_test}")
print(f"Test R^2: {r2_test}")
